# Computing standard thermodynamic properties of species

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 20th, 2022</p>

```{attention}
Always make sure you are using the [latest version of Reaktoro](https://anaconda.org/conda-forge/reaktoro). Otherwise, some new features documented on this website will not work on your machine and you may receive unintuitive errors. Follow these [update instructions](updating_reaktoro_via_conda) to get the latest version of Reaktoro!
```

This tutorial demonstrates the use of Reaktoro for the computation of standard thermodynamic properties of chemical species such as:

* the standard molar Gibbs energy, $G_i^\circ$
* the standard molar Helmholtz energy, $A_i^\circ$
* the standard molar enthalpy, $H_i^\circ$
* the standard molar internal energy, $U_i^\circ$
* the standard molar entropy, $S_i^\circ$
* the standard molar volume, $V_i^\circ$
* the standard molar heat capacity (constant pressure), $C_{P,i}^\circ$
* the standard molar heat capacity (constant volume), $C_{V,i}^\circ$

Let's start with the use of the SUPCRTBL database {cite}`Zimmer2016a` to compute the standard thermodynamic properties of the following chemical species:

* CO{{_2}}(aq)
* CO{{_2}}(g)
* Calcite

In [2]:
from reaktoro import *

db = SupcrtDatabase("supcrtbl")

CO2g    = db.species("CO2(g)")
CO2aq   = db.species("CO2(aq)")
calcite = db.species("Calcite")

We can now use method `props` in the {{Species}} class to compute the standard thermodynamic properties of these species at 60 °C and 100 bar:

In [3]:
print("STANDARD THERMODYNAMIC PROPERTIES OF CO2(G) AT 60 °C AND 100 BAR")
print(CO2g.props(60, "C", 100, "bar"))

print("STANDARD THERMODYNAMIC PROPERTIES OF CO2(AQ) AT 60 °C AND 100 BAR")
print(CO2aq.props(60, "C", 100, "bar"))

print("STANDARD THERMODYNAMIC PROPERTIES OF CALCITE AT 60 °C AND 100 BAR")
print(calcite.props(60, "C", 100, "bar"))

STANDARD THERMODYNAMIC PROPERTIES OF CO2(G) AT 60 °C AND 100 BAR
+------------------------------------------+---------+-------------+
| Property                                 |   Value |        Unit |
+------------------------------------------+---------+-------------+
| Temperature                              |  333.15 |           K |
| Pressure                                 |   1e+07 |          Pa |
| Standard Gibbs Energy                    | -401904 |       J/mol |
| Standard Enthalpy                        | -392186 |       J/mol |
| Standard Volume                          |       0 |      m3/mol |
| Standard Volume (Temperature Derivative) |       0 |  m3/(mol*K) |
| Standard Volume (Pressure Derivative)    |       0 | m3/(mol*Pa) |
| Standard Isobaric Heat Capacity          | 38.5567 |   J/(mol*K) |
| Standard Isochoric Heat Capacity         | 38.5567 |   J/(mol*K) |
| Standard Internal Energy                 | -392186 |       J/mol |
| Standard Entropy                    

Now let's calculate the standard thermodynamic properties of CO{{_2}}(aq) from 25 to 300 °C along the saturation pressure of water. The code block below will build a Python dictionary containing data that we will plot later (i.e., the standard molar Gibbs energy and standard molar enthalpy of the species CO{{_2}}(aq) and the temperatures used to calculate these properties):

In [4]:
import numpy as np

temperatures = np.linspace(25.0, 300.0, 100) + 273.15  # in K

data = { "T": [], "G0": [], "H0": [] }

for T in temperatures:
    P = waterSaturationPressureWagnerPruss(T)   # in Pa
    props = CO2aq.props(T, P)
    data["T" ].append(float(T - 273.15))        # in °C
    data["G0"].append(float(props.G0 * 0.001))  # in kJ/mol
    data["H0"].append(float(props.H0 * 0.001))  # in kJ/mol

```{tip}
You might also be interested in other methods for calculating the thermodynamic properties of water besides `waterSaturationPressureWagnerPruss`, which implements the water saturation pressure equation in {cite:t}`Wagner2002`. Below are other methods available in Reaktoro:

* `waterDensityHGK`
* `waterDensityWagnerPruss`
* `waterLiquidDensityHGK`
* `waterLiquidDensityWagnerPruss`
* `waterVaporDensityHGK`
* `waterVaporDensityWagnerPruss`
* `waterPressureHGK`
* `waterPressureWagnerPruss`
* `waterSaturationPressureWagnerPruss`
* `waterSaturationLiquidDensityWagnerPruss`
* `waterSaturationVapourDensityWagnerPruss`
* `waterThermoPropsHGK`
* `waterThermoPropsWagnerPruss`

Search for these method names in [Reaktoro's API Reference](https://reaktoro.org/api/) to learn more about how to use them.
```

We'll use the [bokeh](https://bokeh.org/) plotting library next. First, we need to import it and initialize it to work with Jupyter Notebooks:

In [5]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

We can now define our interactive plot:

In [12]:
hovertool = HoverTool()
hovertool.tooltips = [("Temperature", "@T °C"), ("G°,CO2(aq)", "@G0 kJ/mol"), ("H°,CO2(aq)", "@H0 kJ/mol")]

p = figure(
    title="STANDARD THERMODYNAMIC PROPERTIES OF CO2(AQ)\nALONG WATER SATURATION PRESSURE", 
    x_axis_label='TEMPERATURE [°C]', 
    y_axis_label=r"",
    sizing_mode="scale_width")

p.add_tools(hovertool)

p.line("T", "G0", source=data, legend_label="G°,CO2(aq)", line_width=5, line_cap="round", line_color="midnightblue")
p.line("T", "H0", source=data, legend_label="H°,CO2(aq)", line_width=5, line_cap="round", line_color="orange")

show(p)

That's it - you can repeat the same process with any other database in Reaktoro (not just SUPCRTBL!). In the next section, we will learn how to calculate the thermodynamic properties of reactions. Keep reading!